# 13-9. 프로젝트 - Movielens 영화 추천 실습
이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.<br>
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.<br>
- 별점을 시청횟수로 해석해서 생각하겠습니다.<br>
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.<br>



In [69]:
import numpy as np
import scipy
import implicit

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.19.5
1.9.3
0.4.8


# 1) 데이터 준비와 전처리
---
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [70]:
import os
import pandas as pd
rating_file_path='./data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [71]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [72]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [73]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [74]:
ratings.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [75]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path='./data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

# 2) 분석해 봅시다.
---
- ratings에 있는 유니크한 영화 개수<br>
- ratings에 있는 유니크한 사용자 수<br>
- 가장 인기 있는 영화 30개(인기순)<br>

In [76]:
# 나중에 csr matrix할때 index가 shape[0]크기를 벗어나는 오류때문에 0부터 시작하도록 미리 변경함
ratings['user_id']=ratings['user_id']-1

In [84]:
ratings['movie_id']=ratings['movie_id']-1

In [79]:
# 나중에 csr matrix할때 index가 shape[0]크기를 벗어나는 오류때문에 0부터 시작하도록 미리 변경함
movies['movie_id']=movies['movie_id']-1

In [85]:
# ratings에 있는 유니크한 영화개수
ratings['movie_id'].nunique()

3628

In [86]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [87]:
# 가장 인기 있느 영화 30개(인기순)
movie_id_user_count = ratings.groupby('movie_id')['user_id'].count()
movie_id_user_count

movie_id
0       2000
1        551
2        339
3        102
4        214
        ... 
3947     752
3948     280
3949      47
3950      36
3951     353
Name: user_id, Length: 3628, dtype: int64

In [88]:
best_movie_id = movie_id_user_count.sort_values(ascending=False) # 큰값이 맨처음 ---> 나중으로 갈수록 값이 작아지도록 정렬

In [89]:
best_movie_id_list=best_movie_id[:5].index.to_list() # 최상위 5개의 index를 series로 변환

In [90]:
best_movie_top5_df = movies[movies['movie_id'].isin(best_movie_id_list)]
best_movie_top5_df

,movie_id,title,genre
257,259,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
1178,1195,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1192,1209,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1959,2027,Saving Private Ryan (1998),Action|Drama|War
2789,2857,American Beauty (1999),Comedy|Drama


In [91]:
best_movie_top5_df=best_movie_top5_df.set_index('movie_id') # 인덱스를 movie_id로 변경한다

In [92]:
best_movie_top5_df.loc[best_movie_id_list] # best_movie_id_list 값 순서와 동일하게 배치된 DataFrame을 출력한다 맨위부터 제일 많이 선호하는영화

,title,genre
movie_id,,
2857,American Beauty (1999),Comedy|Drama
259,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
1195,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1209,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
2027,Saving Private Ryan (1998),Action|Drama|War


In [93]:
best_movie_id_list

[2857, 259, 1195, 1209, 2027]

In [95]:
# ratings에 movie_id로 존재하는 값과 movies에 movie_id로 존재하는 값이 같은 경우 movie의 title 컬럼을 만들고 값을 넣어주려고합니다
ratings = pd.merge(ratings,movies[['movie_id','title']],on='movie_id',how='left')
ratings

,user_id,movie_id,counts,timestamp,title
0,0,1192,5,978300760,One Flew Over the Cuckoo's Nest (1975)
1,0,660,3,978302109,James and the Giant Peach (1996)
2,0,913,3,978301968,My Fair Lady (1964)
3,0,3407,4,978300275,Erin Brockovich (2000)
4,0,2354,5,978824291,"Bug's Life, A (1998)"
...,...,...,...,...,...
836473,6039,1089,3,956715518,Platoon (1986)
836474,6039,1093,5,956704887,"Crying Game, The (1992)"
836475,6039,561,5,956704746,Welcome to the Dollhouse (1995)
836476,6039,1095,4,956715648,Sophie's Choice (1982)


# 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.
---

In [96]:
ratings['user_id'].sort_values(ascending=False) # 나의 id를 선택하기 위해 id번호 맨 마지막 값을 얻어온다 -> 6040

836477    6039
836295    6039
836289    6039
836290    6039
836291    6039
          ... 
33           0
32           0
31           0
30           0
0            0
Name: user_id, Length: 836478, dtype: int64

In [97]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite_movie = ['Contender, The (2000)' , 'Tigerland (2000)' ,'Requiem for a Dream (2000)' ,'Get Carter (1971)' ,'Bootmen (2000)']
my_favorite_movie_id=[3950,3949,3948,3946,3943]

# 저의 'id'는 6039 의 다음 6040 로 하겠습니다.
# id=6040 가 favorite_movie 를 보고 추천을 3으로 주었다고 가정하겠습니다.
my_rate_list = pd.DataFrame({'user_id': [6040]*5, 'movie_id': my_favorite_movie_id, 'counts':[3]*5, 'timestamp':0,'title':my_favorite_movie})

if not ratings.isin({'user_id':[6040]})['user_id'].any():  # user_id에 6041 이라는 데이터가 없다면
    ratings = ratings.append(my_rate_list)                 # 위에 임의로 만든 my_rate_list 데이터를 추가해 줍니다. 

ratings.tail(10) 

,user_id,movie_id,counts,timestamp,title
836473,6039,1089,3,956715518,Platoon (1986)
836474,6039,1093,5,956704887,"Crying Game, The (1992)"
836475,6039,561,5,956704746,Welcome to the Dollhouse (1995)
836476,6039,1095,4,956715648,Sophie's Choice (1982)
836477,6039,1096,4,956715569,E.T. the Extra-Terrestrial (1982)
0,6040,3950,3,0,"Contender, The (2000)"
1,6040,3949,3,0,Tigerland (2000)
2,6040,3948,3,0,Requiem for a Dream (2000)
3,6040,3946,3,0,Get Carter (1971)
4,6040,3943,3,0,Bootmen (2000)


In [122]:
user_id = ratings['user_id'].unique()
movie_id = ratings['title'].unique()
user_to_idx = {v:k for k,v in enumerate(user_id)}
movie_to_idx = {v:k for k,v in enumerate(movie_id)}

In [125]:
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx 을 통해  title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

user_id column indexing OK!!
title column indexing OK!!


In [129]:
ratings.drop('movie_id',axis=1,inplace=True)

In [131]:
ratings.drop('timestamp',axis=1,inplace=True)

In [133]:
ratings.rename(columns={'title':'movie_id'},inplace=True)

In [134]:
ratings

,user_id,counts,movie_id
0,0,5,0
1,0,3,1
2,0,3,2
3,0,4,3
4,0,5,4
...,...,...,...
0,6039,3,1099
1,6039,3,2398
2,6039,3,1318
3,6039,3,2691


# 4) CSR matrix를 직접 만들어 봅시다.
---

In [140]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련.
---

In [141]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [142]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [143]:
# 모델 훈련
als_model.fit(csr_data_transpose)

100%|██████████| 15/15 [00:03<00:00,  4.05it/s]


In [144]:
als_model.item_factors.shape

(3628, 100)

# 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [146]:
# 내가선호하는 5가지 영화중 하나
my_user_idx, requiem_movie_idx = user_to_idx[6040], movie_to_idx['Requiem for a Dream (2000)']

In [149]:
my_vector = als_model.user_factors[my_user_idx]
requiem_vector = als_model.item_factors[requiem_movie_idx]

In [150]:
# 선호도가 낮게 나오네요^^
np.dot(my_vector, requiem_vector)

0.27203968

In [152]:
# 제가 선택하지 않은 영화 
ab_movie_idx = movie_to_idx['American Beauty (1999)']
ab_vector = als_model.item_factors[ab_movie_idx]

In [153]:
# 선호도가 더 낮게 나오네요^^
np.dot(my_vector, ab_vector)

-0.050437592

# 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [154]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
def get_similar_movie(movie_name: str):
    movie_idx = movie_to_idx[movie_name]
    similar_movies = als_model.similar_items(movie_idx)
    similar_movie_list = [idx_to_movie[i[0]] for i in similar_movies]
    return similar_movie_list

In [156]:
get_similar_movie('Tigerland (2000)')

['Tigerland (2000)',
 'Yards, The (1999)',
 'Wisdom of Crocodiles, The (a.k.a. Immortality) (2000)',
 'Under Suspicion (2000)',
 'End of the Affair, The (1955)',
 'Following (1998)',
 'Two Family House (2000)',
 'Splendor (1999)',
 'Jeanne and the Perfect Guy (Jeanne et le garçon formidable) (1998)',
 'Shadow Conspiracy (1997)']

# 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [157]:
movie_recommended = als_model.recommend(my_user_idx, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(755, 0.35434943),
 (1853, 0.30043364),
 (461, 0.29534772),
 (1814, 0.2777233),
 (486, 0.19206962),
 (1887, 0.17959005),
 (1852, 0.17035298),
 (859, 0.14803858),
 (1470, 0.1204398),
 (2056, 0.12015468),
 (1465, 0.11975765),
 (2537, 0.11627371),
 (1857, 0.11292884),
 (1117, 0.10323324),
 (2027, 0.10145312),
 (468, 0.09703316),
 (2243, 0.09660561),
 (2235, 0.0956245),
 (463, 0.08728663),
 (1808, 0.08594224)]

In [158]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Almost Famous (2000)',
 'Best in Show (2000)',
 'Meet the Parents (2000)',
 'Nurse Betty (2000)',
 'Remember the Titans (2000)',
 'Wonder Boys (2000)',
 'Dancer in the Dark (2000)',
 'Cell, The (2000)',
 'Tao of Steve, The (2000)',
 'Way of the Gun, The (2000)',
 'Saving Grace (2000)',
 'Girlfight (2000)',
 'Bring It On (2000)',
 'Virgin Suicides, The (1999)',
 'Croupier (1998)',
 'Magnolia (1999)',
 'Yards, The (1999)',
 'Original Kings of Comedy, The (2000)',
 'Frequency (2000)',
 'Chuck & Buck (2000)']